In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "plantae.settings")
import django
django.setup()

In [2]:
from agent.langgraph.tools import get_cart_items, add_to_cart, search_product
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from dotenv import load_dotenv

load_dotenv()

# LLMs
llm = ChatOpenAI(model="gpt-4.1-nano-2025-04-14", temperature=0.7)
web_search = TavilySearch(max_results=2)


In [4]:
from carts.models import CartItem
@tool
def get_cart_items(user_id: int) -> str:
    """
    Get the products in the cart by using user id.
    """
    items = CartItem.objects.filter(user_id=user_id)
    if not items.exists():
        return "Your cart is empty."
    return "\n".join([f"{item.product.product_name} × {item.quantity}" for item in items])

In [10]:
get_cart_items.invoke({"user_id": 1})

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [20]:
from carts.models import CartItem, Product, Variation
from asgiref.sync import sync_to_async
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

@tool
def get_cart_items(user_id: int) -> str:
    """
    Get the products in the cart by using user id.
    """
    items = CartItem.objects.select_related('product').filter(user_id=user_id)
    if not items.exists():
        return "Your cart is empty."
    return "\n".join([f"{item.product.product_name} × {item.quantity}" for item in items])


In [21]:
result = get_cart_items.invoke({"user_id": 1})
print(result)

Planter × 1
Adenium × 1


In [37]:
@tool
def search_product(product_name: str) -> str:
    """
    Search for a product by name. Returns product information including ID and available variations.
    """
    try:
        products = Product.objects.filter(product_name__icontains=product_name)
        if not products.exists():
            return f"No products found matching '{product_name}'"
        
        result = []
        for product in products:
            variations = Variation.objects.filter(product=product)
            variation_info = []
            for var in variations:
                variation_info.append(f"{var.variation_category}: {var.variation_value}")
            
            product_info = f"Product ID: {product.id}, Name: {product.product_name}"
            if variation_info:
                product_info += f", Available variations: {', '.join(variation_info)}"
            result.append(product_info)
        
        return "\n".join(result)
    except Exception as e:
        return f"Error searching for product: {str(e)}"
    
result = search_product.invoke({"product_name": 'Stone Grey 22 inch planter'})
print(result)

No products found matching 'Stone Grey 22 inch planter'


In [32]:
from django.contrib.auth import get_user_model

@tool
def add_to_cart(user_id: int, product_name: str, variation_dict: dict = None) -> str:
    """
    Add the product to the cart by product name. If there exists a variation in the product, first get the variations THEN ONLY add the product with variation in the cart. 
    If the product or product with certain variation already exists in the cart, increase the quantity of that product by 1.
    """
    if variation_dict is None:
        variation_dict = {}
    try:
        User = get_user_model()
        current_user = User.objects.get(id=user_id)
        
        # Search for product by name
        products = Product.objects.filter(product_name__icontains=product_name)
        if not products.exists():
            return f"No product found with name '{product_name}'. Please check the product name and try again."
        
        if products.count() > 1:
            product_list = "\n".join([f"- {p.product_name} (ID: {p.id})" for p in products])
            return f"Multiple products found with similar names. Please be more specific:\n{product_list}"
        
        product = products.first()
        product_variation = []

        # Extract variations
        for key, value in variation_dict.items():
            try:
                variation = Variation.objects.get(
                    product=product,
                    variation_category__iexact=key,
                    variation_value__iexact=value
                )
                product_variation.append(variation)
            except Variation.DoesNotExist:
                continue

        cart_items = CartItem.objects.filter(product=product, user=current_user)

        for item in cart_items:
            existing_variation = list(item.variation.all())
            if set(existing_variation) == set(product_variation):
                item.quantity += 1
                item.save()
                return f"Increased quantity of {product.product_name} with selected variations."

        # If no matching variation, create new item
        new_item = CartItem.objects.create(product=product, quantity=1, user=current_user)
        if product_variation:
            new_item.variation.set(product_variation)
        new_item.save()
        return f"Added {product.product_name} to cart."

    except User.DoesNotExist:
        return "User not found."
    except Exception as e:
        return f"Error: {str(e)}"
    

result = add_to_cart.invoke({"user_id": 1,
                             "product_name": "ayush",
                             "variation_dict": {"color": "Stone Grey", "size": "24 inch"}
                             })
print(result)

No product found with name 'ayush'. Please check the product name and try again.


In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

cart_agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4.1-nano-2025-04-14", temperature=0.7),
    tools=[get_cart_items, add_to_cart, search_product],
    prompt="""You are a helpful plant store assistant. You can help users with:
1. Checking their cart contents
2. Adding products to their cart (The query of product may contain variations. From the query, you need to extract the product name and the variations)
3. Searching for products
4. Providing information about plants

Always be friendly and helpful. When users ask about their cart, use the get_cart_items tool.
When they want to add products, first  use the add_to_cart tool.
When they want to search for products, use the search_product tool.

Remember to include the user_id when using cart-related tools."""
)

# Test the agent with cart query
inputs = {
    "messages": [
        HumanMessage(content="Add a Planter (Stone Grey 22 inch) to my cart I have user_id = 1")
    ]
}

# Run the agent
#result = react_agent.invoke(inputs)

# Print the result
print("Agent Response:")
print(result["messages"][-1].content)

# You can also stream the response to see the tool calls in action
print("\n" + "="*50)
print("Streaming response to see tool calls:")
for chunk in cart_agent.stream(inputs, stream_mode="updates"):
    print(chunk)

Agent Response:
Petunias are popular flowering plants that need proper care to thrive. They require regular watering—about 1 to 2 inches per week, more in hot weather—and prefer well-drained soil. They thrive in full sunlight, ideally 5-6 hours of direct sun daily. For detailed tips on pruning, fertilizing, and disease management, you can check [this guide](https://garvillo.com/how-to-care-for-petunias/).

Streaming response to see tool calls:
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HjA9V5uURnbBb8mZ2dJ38MQt', 'function': {'arguments': '{"product_name":"Planter (Stone Grey 22 inch)"}', 'name': 'search_and_check_variations'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 358, 'total_tokens': 384, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_

In [43]:
web_search = TavilySearch(max_results=3)

# Create the research agent
research_agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4.1-nano-2025-04-14", temperature=0.7),
    tools=[web_search],
    prompt="""You are a plant research assistant. 
You answer any questions about plant care, watering frequency, soil type, nutrients, sunlight, pests, diseases, and any other plant-related information.
Always use the web_search tool to provide up-to-date and accurate information.
Be concise, friendly, and cite your sources if possible."""
)

# Example usage: ask a plant care question
inputs = {
    "messages": [
        HumanMessage(content="I want to know about petunia plant")
    ]
}

# Run the agent
result = research_agent.invoke(inputs)

# Print the result
print("Research Agent Response:")
print(result["messages"][-1].content)

# (Optional) Stream the response to see tool calls in action
print("\n" + "="*50)
print("Streaming response to see tool calls:")
for chunk in research_agent.stream(inputs, stream_mode="updates"):
    print(chunk)

Research Agent Response:
Petunias are popular flowering plants that need proper care to thrive. They require regular watering—about 1 to 2 inches per week, more in hot weather—and prefer well-drained soil. They thrive in full sunlight, ideally 5-6 hours of direct sun daily. For detailed tips on pruning, fertilizing, and disease management, you can check [this guide](https://garvillo.com/how-to-care-for-petunias/).

Streaming response to see tool calls:
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DVkOo8MOrrgCPnIqx9wo9il5', 'function': {'arguments': '{"query":"petunia plant care, watering, sunlight, soil, pests, and diseases"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 839, 'total_tokens': 869, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_

In [47]:
# Fixed Supervisor Agent - Using proper LangGraph node structure
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Literal

# Define the state schema
class SupervisorState(TypedDict):
    messages: Annotated[list, add_messages]
    agent_type: str

# Create the supervisor LLM
supervisor_llm = ChatOpenAI(model="gpt-4.1-nano-2025-04-14", temperature=0.3)

def supervisor_node(state: SupervisorState) -> SupervisorState:
    """Supervisor node that decides which agent to route to"""
    messages = state["messages"]
    
    # Create a decision prompt as a system message
    system_prompt = """You are a supervisor that routes user queries to the appropriate agent.

Available agents:
1. CART_AGENT - For cart operations, adding products, checking cart contents, searching store products
2. RESEARCH_AGENT - For plant care questions, watering, soil, nutrients, pests, diseases, general plant information

Analyze the user's intent and respond with ONLY one word:
- "cart" if the user wants to interact with their cart, add products, or search store products
- "research" if the user is asking about plant care, care instructions, or general plant information

Respond with only the word:"""

    # Create the message list with system prompt and user message
    decision_messages = [
        SystemMessage(content=system_prompt),
        messages[-1]  # The user's message
    ]
    
    # Get supervisor decision
    response = supervisor_llm.invoke(decision_messages)
    agent_decision = response.content.strip().lower()
    
    # Validate decision
    if agent_decision not in ["cart", "research"]:
        agent_decision = "research"  # Default to research
    
    return {
        "messages": messages,
        "agent_type": agent_decision
    }

# Create the supervisor graph
def create_supervisor_agent():
    """Create a supervisor agent that routes between cart and research agents"""
    
    workflow = StateGraph(SupervisorState)
    
    # Add nodes
    workflow.add_node("supervisor", supervisor_node)
    workflow.add_node("cart_agent", cart_agent)
    workflow.add_node("research_agent", research_agent)
    
    # Set entry point
    workflow.set_entry_point("supervisor")
    
    # Add conditional edges
    def route_to_agent(state: SupervisorState) -> Literal["cart_agent", "research_agent", "__end__"]:
        """Route to appropriate agent based on supervisor decision"""
        agent_type = state.get("agent_type", "")
        if agent_type == "cart":
            return "cart_agent"
        elif agent_type == "research":
            return "research_agent"
        else:
            return END
    
    workflow.add_conditional_edges("supervisor", route_to_agent)
    workflow.add_edge("cart_agent", END)
    workflow.add_edge("research_agent", END)
    
    return workflow.compile()

# Create the supervisor agent
supervisor_agent = create_supervisor_agent()

# Test the supervisor agent
def test_supervisor(query: str):
    """Test the supervisor with different types of queries"""
    inputs = {
        "messages": [HumanMessage(content=query)],
        "agent_type": ""
    }
    
    print(f"Query: {query}")
    print("-" * 50)
    
    result = supervisor_agent.invoke(inputs)
    
    # Print the final response
    final_message = result["messages"][-1]
    print(f"Final Response: {final_message.content}")
    
    # Print which agent was used
    agent_type = result.get("agent_type", "unknown")
    print(f"Agent Used: {agent_type.upper()}")
    print("=" * 50)
    print()

# Test different types of queries
test_queries = [
    "What's in my cart? I have user_id = 1",
    "Add an adenium plant to my cart",
    "What soil type is best for succulents?",
]

for query in test_queries:
    test_supervisor(query)

Query: What's in my cart? I have user_id = 1
--------------------------------------------------
Final Response: Your cart contains:
- 1 Adenium
- 1 Maize Seeds
- 2 Planters

Would you like to add more items or need any other assistance?
Agent Used: CART

Query: Add an adenium plant to my cart
--------------------------------------------------
Final Response: The Adenium plant has been added to your cart. Would you like to see your cart contents or need help with anything else?
Agent Used: CART

Query: What soil type is best for succulents?
--------------------------------------------------
Final Response: The best soil for succulents is a well-draining mix. A common recommendation is a 1:1 ratio of succulent soil to inorganic grit like perlite or pumice. Organic matter such as pine bark, coconut coir, or compost can also be included, along with mineral options like coarse sand or volcanic rock. The key is good drainage to prevent root rot. [Sources: Reddit, Mountain Crest Gardens, Arti

NameError: name 'workflow' is not defined

In [55]:
from category.models import Category
from store.models import Product
def fetch_products_by_category(category_name: str) -> list:
    """Fetch products by category using Django ORM"""
    try:
        category = Category.objects.get(category_name__iexact=category_name)
        products = Product.objects.filter(category=category)
        return list(products)
    except Category.DoesNotExist:
        return []

fetch_products_by_category('seeds')

[<Product: Maize Seeds>, <Product: Peanut Seeds>]